In [2]:
import pandas as pd
from obspy import UTCDateTime
from obspy.clients.fdsn import Client as FDSNclient
from packages/filtering import filtering_data
from fourier import fourier_transform
from core import main_processing
from obspy.clients.seedlink.easyseedlink import create_client
from obspy import Trace, UTCDateTime, Stream
from obspy.clients.fdsn import Client as FDSNclient
from obspy.core.inventory import Inventory
import matplotlib.pyplot as plt
import numpy as np
import logging

def process_index(element, ref_time):
    event_stat = {}
    try:
    
        url = 'rtserve.iris.washington.edu:18000'
        source = {
            "LX": {"MORF": ["BHE", "BHN", "BHZ"]}
        }

        loc = '--'
        
        client = FDSNclient("IRIS")
        gap = 120
        
        ref_time = UTCDateTime(ref_time)

        starttime = ref_time - gap  # 2 minutes before
        endtime = ref_time + gap  # 2 minutes after

        st = None
        
        # Attempt to print available streams
        for server, values in source.items():
            for channel, ch in values.items():
                for i in ch:
                    trace = client.get_waveforms(server, channel, loc, i, starttime, endtime, attach_response=True)
                    
                    if st is None:
                        st = trace
                    else:
                        st += trace  # Use += to concatenate traces

        if st is None:
            logging.error("No data retrieved for the given time window.")
            return {}
        
        st2 = st.copy()

        # Process components
        st_z = st.select(component="Z")
        st_n = st.select(component="N")
        st_e = st.select(component="E")

        # Process Z component
        if len(st_z) > 0:
            filtered_z, time_vector_z = filtering_data(st2.select(component="Z")[0])
            dom_freq_z, amp_z = fourier_transform(filtered_z, st_z[0])
            event_stat["Domin_freq_z"] = dom_freq_z
        else:
            print("No Z component traces available.")

        # Process N component
        if len(st_n) > 0:
            filtered_n, time_vector_n = filtering_data(st2.select(component="N")[0])
            dom_freq_n, amp_n = fourier_transform(filtered_n, st_n[0])
            event_stat["Domin_freq_n"] = dom_freq_n
        else:
            print("No N component traces available.")

        # Process E component
        if len(st_e) > 0:
            filtered_e, time_vector_e = filtering_data(st2.select(component="E")[0])
            dom_freq_e, amp_e = fourier_transform(filtered_e, st_e[0])
            event_stat["Domin_freq_e"] = dom_freq_e
        else:
            print("No E component traces available.")

        details = main_processing(filtered_z, gap, dom_freq_z)
        display(details)

        for key, dicts in details.items():
            if key == 1:
                for stat, values in dicts.items():
                    if stat == "Vel Amp (m/s)":
                        event_stat["P-Vel amp (m/s)"] = values
                    elif stat == "Disp Amp (m)":
                        event_stat["P-Disp amp (m)"] = values
                    elif stat == "peak2peak":
                        event_stat["P-peak2peak"] = values
                    elif stat == "r":
                        event_stat["P-r"] = values
                    elif stat == "moment_history":
                        event_stat["P-moment_history"] = values
                    elif stat == "tau_c":
                        event_stat["P-tau_c"] = values    
                    else:
                        print(f"{key} error")
            elif key == 2:
                for stat, values in dicts.items():
                    if stat == "Vel Amp (m/s)":
                        event_stat["S-Vel amp (m/s)"] = values
                    elif stat == "Disp Amp (m)":
                        event_stat["S-Disp amp (m)"] = values
                    elif stat == "peak2peak":
                        event_stat["S-peak2peak"] = values
                    elif stat == "r":
                        event_stat["S-r"] = values
                    elif stat == "moment_history":
                        event_stat["S-moment_history"] = values
                    elif stat == "tau_c":
                        event_stat["S-tau_c"] = values    
                    else:
                        print(f"{key} error")
            elif key == "RMS":
                event_stat["RMS"] = dicts    
            elif key == "Energy":
                event_stat["Energy"] = dicts 
            elif key == "peak_freq":
                event_stat["peak_freq"] = dicts 
            elif key == "wavelength":
                event_stat["wavelength"] = dicts 
            elif key == "peak_disp":
                event_stat["peak_disp"] = dicts 
            elif key == "M0":
                event_stat["M0"] = dicts 
            else:
                event_stat["other infos"] = values
                
        print(f"...Finished processing {element}")    
        return event_stat
    except Exception as e:
        print(f"Error fetching index: {element}, Error: {e}")

SyntaxError: invalid syntax (2043899793.py, line 4)

In [ ]:
import numpy as np
from filtering import filtering_data
from fourier import fourier_transform
from core import main_processing
import joblib

# Load the model from the .pkl file
model = joblib.load('model_FINAL.pkl')
best_model = model.best_estimator_

#events: 

event = {"1": "2024-11-06 16:03:17" # M3.4 "close" sensed-n = 0
         ,"2": "2024-09-04 00:03:00" # M1.9 "close" int = II sensed-n = 1
         ,"3": "2024-06-29 04:45:00" # M3.4         III/IV sensed-n = 1
         ,"5": "2024-08-26 05:11:00" # M5.3 "close" int IV/V sensed-n = 1
         ,"4": "2024-08-11 07:18:00" # M2.6 "close" III sensed-n = 1
         ,"6": "2024-08-11 05:48:00" # M3.4 "close" III sensed-n = 1
        }

# pre-processing

event_data = pd.DataFrame()
for key, values in event.items():
    event_stat = process_index(key, values)
    event_data["datetime"] = values
    event_data = pd.concat([event_data, pd.Series(event_stat, name=key).to_frame().T], ignore_index=True, axis=0)
    # Reset index to have a clean DataFrame
    event_data.reset_index(drop=True, inplace=True)
    
event_data["Domin_freq_n"] = abs(event_data["Domin_freq_n"])
event_data["Domin_freq_e"] = abs(event_data["Domin_freq_e"])
event_data["Domin_freq_z"] = abs(event_data["Domin_freq_z"])
event_data["P-Vel amp (m/s)"] = abs(event_data["P-Vel amp (m/s)"])
event_data["S-Vel amp (m/s)"] = abs(event_data["S-Vel amp (m/s)"])
event_data["P-Disp amp (m)"] = abs(event_data["P-Disp amp (m)"])
event_data["S-Disp amp (m)"] = abs(event_data["S-Disp amp (m)"])

# select the features
features = event_data [['Domin_freq_z','P-peak2peak', 'P-Disp amp (m)', 'P-r', 'P-moment_history', 'P-tau_c','RMS','Energy']]

from sklearn.preprocessing import MinMaxScaler

# normalize
normalizer = MinMaxScaler()
normalizer.fit(features)
features_norm = normalizer.transform(features)
features_norm = pd.DataFrame(features_norm, columns=features.columns)

for index, event in features_norm.iterrows():      
    y_prob = best_model.predict_proba([event])[:, 1]  # Probabilities for the positive class
    custom_threshold = 0.01
    y_pred_custom_threshold = (y_prob >= custom_threshold).astype(int)
    print(y_pred_custom_threshold)
    if y_pred_custom_threshold == 1:
        print("ALERT!")             